**Processo ETL - Desafio DIO**

ETL de dados do TMDb (The Movie Database) através de uso de API Pública.

-> Extração: Os dados dos filmes mais popularese segundo o TMDb serão extraídos por um API pública e carregada em Dataframe pandas.

-> Transformação: Será feita uma limpeza simples dos dados, deixando apenas os dados importanres para análise assim como a classificação das avaliações por faixas de acordo com a coluna *vote_average*

-> Carga (Load): Os dados tratados serão carregados em um arquivo parquet.

In [1]:
# Bibliotecas
import requests
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

***Extração dos Dados da Api do TMDb***

In [2]:
# Chave da API do TMDb
api_key = 'api_key_tmdb'

# Obtendo os dados de filmes populares
url = f'https://api.themoviedb.org/3/movie/popular?api_key={api_key}&language=pt-BR&page=1'
response = requests.get(url)

# Verificando se a solicitação foi bem-sucedida
if response.status_code == 200:
    data = response.json()
    # Extraindo os resultados para um DataFrame
    df = pd.DataFrame(data['results'])
else:
    print('Erro ao acessar a API do TMDb')

In [3]:
# Conhecendo os dados retornados
df.head()

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/xFYpUmB01nswPgbzi8EOCT1ZYFu.jpg,"[12, 28, 18]",980489,en,Gran Turismo,"Baseado na história de Jann Mardenborough, e r...",2933.587,/qU60nhBRbKU23gIGrZi2CvUj6MQ.jpg,2023-08-09,Gran Turismo: De Jogador a Corredor,False,8.0,806
1,False,/pA3vdhadJPxF5GA1uo8OPTiNQDT.jpg,"[28, 18]",678512,en,Sound of Freedom,"A história de Tim Ballard, um ex-agente do gov...",2623.519,/2BWhcrlPr6UvSJy5D044Ikq2QxH.jpg,2023-07-03,Som da Liberdade,False,8.0,598
2,False,/mRGmNnh6pBAGGp6fMBMwI8iTBUO.jpg,"[27, 9648, 53]",968051,en,The Nun II,Quatro anos após os acontecimentos na abadia d...,2614.527,/omV2IW2OlFTSw6Hih13hz6lFdvP.jpg,2023-09-06,A Freira 2,False,6.9,497
3,False,/1syW9SNna38rSl9fnXwc9fP7POW.jpg,"[28, 878, 12]",565770,en,Blue Beetle,"Jaime Reyes, um adolescente de origem mexicana...",1851.699,/y5sdkgO4IJMTnkjh9PG7kREPWrP.jpg,2023-08-16,Besouro Azul,False,7.1,1147
4,False,/8pjWz2lt29KyVGoq1mXYu6Br7dE.jpg,"[28, 878, 27]",615656,en,Meg 2: The Trench,Um mergulho exploratório nas profundezas do oc...,1177.938,/klGKGITBYYyTiHrph1VDSgGULOR.jpg,2023-08-02,Megatubarão 2,False,6.9,2121


***Transformação dos dados***

In [4]:
# Entendendo os dados
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   adult              20 non-null     bool   
 1   backdrop_path      20 non-null     object 
 2   genre_ids          20 non-null     object 
 3   id                 20 non-null     int64  
 4   original_language  20 non-null     object 
 5   original_title     20 non-null     object 
 6   overview           20 non-null     object 
 7   popularity         20 non-null     float64
 8   poster_path        20 non-null     object 
 9   release_date       20 non-null     object 
 10  title              20 non-null     object 
 11  video              20 non-null     bool   
 12  vote_average       20 non-null     float64
 13  vote_count         20 non-null     int64  
dtypes: bool(2), float64(2), int64(2), object(8)
memory usage: 2.0+ KB


In [5]:
df.describe()

,id,popularity,vote_average,vote_count
count,2.000000e+01,20.0000,20.000000,20.000000
mean,8.353688e+05,1116.1276,7.120000,1123.500000
std,2.322527e+05,753.5805,0.768868,1503.394562
min,3.466980e+05,555.1330,4.400000,5.000000
25%,6.757685e+05,655.9600,6.900000,76.000000
50%,8.894145e+05,788.1235,7.150000,458.500000
75%,9.859512e+05,1049.2485,7.550000,1390.500000
max,1.172009e+06,2933.5870,8.000000,5203.000000


In [6]:
# Limpeza dos dados
# Matendo apenas as colunas que são interessantes para análise
df_movie = df.drop(['backdrop_path',	'id', 'poster_path',	'release_date', 'video'], axis=1)
df_movie.head()

,adult,genre_ids,original_language,original_title,overview,popularity,title,vote_average,vote_count
0,False,"[12, 28, 18]",en,Gran Turismo,"Baseado na história de Jann Mardenborough, e r...",2933.587,Gran Turismo: De Jogador a Corredor,8.0,806
1,False,"[28, 18]",en,Sound of Freedom,"A história de Tim Ballard, um ex-agente do gov...",2623.519,Som da Liberdade,8.0,598
2,False,"[27, 9648, 53]",en,The Nun II,Quatro anos após os acontecimentos na abadia d...,2614.527,A Freira 2,6.9,497
3,False,"[28, 878, 12]",en,Blue Beetle,"Jaime Reyes, um adolescente de origem mexicana...",1851.699,Besouro Azul,7.1,1147
4,False,"[28, 878, 27]",en,Meg 2: The Trench,Um mergulho exploratório nas profundezas do oc...,1177.938,Megatubarão 2,6.9,2121


In [7]:
# Função para classificar as avaliações
def classificar_avaliacao(vote_average):
    if vote_average <= 3.9:
        return 'Baixa'
    elif vote_average <= 7.0:
        return 'Média'
    else:
        return 'Alta'


In [8]:
# Criando a coluna classificacao, a partir da aplicação da função classificar_avaliacao à coluna 'vote_average'
df_movie['classificacao'] = df_movie['vote_average'].apply(classificar_avaliacao)
# Exibindo os dados
df_movie.head()

,adult,genre_ids,original_language,original_title,overview,popularity,title,vote_average,vote_count,classificacao
0,False,"[12, 28, 18]",en,Gran Turismo,"Baseado na história de Jann Mardenborough, e r...",2933.587,Gran Turismo: De Jogador a Corredor,8.0,806,Alta
1,False,"[28, 18]",en,Sound of Freedom,"A história de Tim Ballard, um ex-agente do gov...",2623.519,Som da Liberdade,8.0,598,Alta
2,False,"[27, 9648, 53]",en,The Nun II,Quatro anos após os acontecimentos na abadia d...,2614.527,A Freira 2,6.9,497,Média
3,False,"[28, 878, 12]",en,Blue Beetle,"Jaime Reyes, um adolescente de origem mexicana...",1851.699,Besouro Azul,7.1,1147,Alta
4,False,"[28, 878, 27]",en,Meg 2: The Trench,Um mergulho exploratório nas profundezas do oc...,1177.938,Megatubarão 2,6.9,2121,Média


***Etapa de Carga dos dados***

In [9]:
# Salvando o DataFrame em um arquivo Parquet
pq.write_table(pa.Table.from_pandas(df_movie), 'filmes_populares_tmdb.parquet')

In [10]:
# Conferindo os dados carregados em parquet
pq.read_table('filmes_populares_tmdb.parquet')

pyarrow.Table
adult: bool
genre_ids: list<item: int64>
  child 0, item: int64
original_language: string
original_title: string
overview: string
popularity: double
title: string
vote_average: double
vote_count: int64
classificacao: string
----
adult: [[false,false,false,false,false,...,false,false,false,false,false]]
genre_ids: [[[12,28,18],[28,18],...,[53],[28,12,878]]]
original_language: [["en","en","en","en","en",...,"en","te","fr","is","en"]]
original_title: [["Gran Turismo","Sound of Freedom","The Nun II","Blue Beetle","Meg 2: The Trench",...,"Elemental","అమిగోస్","Seule : Les dossiers Silvercloud","Napóleonsskjölin","Transformers: Rise of the Beasts"]]
overview: [["Baseado na história de Jann Mardenborough, e relata a trajetória de um jogador de videogame que utilizou suas habilidades nos simuladores para se tornar um piloto profissional de verdade.","A história de Tim Ballard, um ex-agente do governo dos EUA, que largou seu emprego para dedicar sua vida a resgatar crianças de tra